In [13]:
!pip install TTS
!pip install pyarrow

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
from TTS.api import TTS # needs Python 3.11 or older
from ask_lisa import ask_lisa

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

/opt/conda/lib/python3.10/site-packages/librosa/core/intervals.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [25]:
base_dir = "asr_bundestag_clean"
train = base_dir + "/train_nodev"
validate = base_dir + "/train_dev"
test = base_dir + "/test"

In [26]:
mapping = {}

with open(train + "/wav.scp", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split()
        mapping.update({words[0]: [words[1]]})

for i, (key, value) in enumerate(mapping.items()):
    if i >= 5:
        break
    print(key, ": ", value)

7535988_0_172_12 :  ['wavs/7535988_0_172_12.wav']
7536262_0_15_7 :  ['wavs/7536262_0_15_7.wav']
7546736_0_163_2 :  ['wavs/7546736_0_163_2.wav']
7511688_0_142_16 :  ['wavs/7511688_0_142_16.wav']
7535129_3_253 :  ['wavs/7535129_3_253.wav']


In [18]:
with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split(maxsplit=1)
        mapping[words[0]].append(words[1])

for i, (key, value) in enumerate(mapping.items()):
    if i >= 2:
        break
    print(key, ": ", value)

7535988_0_172_12 :  ['wavs/7535988_0_172_12.wav', 'das bedeutet jetzt dass die länder auf jeden fall daten erheben müssen']
7536262_0_15_7 :  ['wavs/7536262_0_15_7.wav', 'kein geschenk das ist verdient meine damen und herren']


In [19]:
word_count = {}
with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        count = len(line.strip().split()) - 1
        if count in word_count:
            word_count[count] = word_count[count] + 1
        else:
            word_count[count] = 1

counts = sorted(word_count.keys())
frequencies = [word_count[c] for c in counts]

plt.bar(counts, frequencies)
plt.xlabel("Wörter pro Transkript")
plt.ylabel("Häufigkeit")
plt.show()

In [20]:
filtered_mapping = {}
for key, value in mapping.items():
    count = len(value[1].split())
    if 70 < count < 1000:
        filtered_mapping.update({key: value})

mapping = filtered_mapping

for i, (key, value) in enumerate(mapping.items()):
    if i >= 5:
        break
    print(key, ": ", value)

print(len(mapping.items()))

7506424_2_69 :  ['wavs/7506424_2_69.wav', 'cdu csu fraktion frau präsidentin meine sehr geehrten damen und herren liebe kolleginnen und kollegen wo man am meisten drauf erpicht grad das bekommt man meistens nicht liebe kolleginnen und kollegen von den linken und den grünen dieses zitat von wilhelm busch passt perfekt zu ihren anträgen gern erläutere ich warum das so ist sie fordern dass die co kosten allein den vermieterinnen und vermietern aufgebürdet werden sollen und erhoffen sich hierdurch dass die energetische sanierung der gebäude angereizt wird']
7520734_0_99_1 :  ['wavs/7520734_0_99_1.wav', 'hat kann man davon ausgehen dass die regulierung der europäischen union egal ob das jetzt eine verordnung oder eine direktive wird dieselbe richtung gehen wird sie wird detaillierter sein sie wird bestimmte dinge nochmal ausführlicher auch sicher ausbuchstabieren ich halte das deutsche gesetz für eine gute vorbereitung und letzten endes gibt es den deutschen unternehmen da auch ein stück we

In [21]:
import json
from ask_lisa import ask_lisa

def generate_qa(transcript, difficulty):
    instruction = f"""
    Erstelle aus dem folgenden Kontext Frage-Antwort-Paare für drei Komplexitätsstufen.

    - Jede Antwort soll präzise und kurz sein. Maximal 3 Worte. Besser 1-2 Worte.
    - Gib das Ergebnis ausschließlich als JSON-Objekt mit den Keys "level_1", "level_2" und "level_3" aus.
    - Unter jedem Key steht wiederum ein JSON-Objekt mit "question" und "answer".
    - Keine anderen Texte oder Erklärungen ausgeben.

    Definition der Komplexitätsgrade:
    1 = sehr einfache, faktische Frage  
    2 = inhaltlich tiefergehende, aber kurze Frage  
    3 = analytische oder interpretierende Frage (z. B. Ursache, Wirkung, Ziel)

    Kontext:
    {transcript}

    Beispielausgabe:
    {{
      "level_1": {{ "question": "Wer wird angesprochen?", "answer": "Linke und Grüne" }},
      "level_2": {{ "question": "Was wird gefordert?", "answer": "CO-Kosten übernehmen" }},
      "level_3": {{ "question": "Warum sollen Vermieter zahlen?", "answer": "Sanierungsanreiz" }}
    }}
    """

    qa_res = ask_lisa(instruction, transcript)

    try:
        qa_all = json.loads(qa_res)
    except json.JSONDecodeError:
        print("⚠️ Antwort war kein gültiges JSON:", qa_res)
        return None, None, difficulty

    key_map = {1: "level_1", 2: "level_2", 3: "level_3"}
    key = key_map.get(difficulty)

    qa = qa_all.get(key, {})
    question = qa.get("question", "")
    answer = qa.get("answer", "")

    return question, answer, difficulty


In [8]:
generate_qa("es gibt zahlreiche nicht abklingende proteste es gibt initiativen es gibt self made projekte petitionen umfragen zeigen dass große teile der gesellschaft bereit sind jetzt wo wir hier reden haben wir vor dem bundestag eine begleitende kundgebung mit über zehn bewegungen und es gibt jetzt gerade redebeiträge und die zeigen ihnen auch wieder dass es nicht nur diese petition ist sondern dass es hunderttausende wenn nicht sogar millionen von menschen sind die tagtäglich immer wieder dafür kämpfen dass hier endlich was passiert im bereich der mobilitäts und verkehrswende zum schluss", 1)

Response status code: 200


('Was wird gefordert?', 'Verkehrswende', 1)

In [9]:
generate_qa("es gibt zahlreiche nicht abklingende proteste es gibt initiativen es gibt self made projekte petitionen umfragen zeigen dass große teile der gesellschaft bereit sind jetzt wo wir hier reden haben wir vor dem bundestag eine begleitende kundgebung mit über zehn bewegungen und es gibt jetzt gerade redebeiträge und die zeigen ihnen auch wieder dass es nicht nur diese petition ist sondern dass es hunderttausende wenn nicht sogar millionen von menschen sind die tagtäglich immer wieder dafür kämpfen dass hier endlich was passiert im bereich der mobilitäts und verkehrswende zum schluss", 2)

Response status code: 200


('Wie zeigt sich der Druck?', 'Proteste und Kundgebungen', 2)

In [11]:
generate_qa("es gibt zahlreiche nicht abklingende proteste es gibt initiativen es gibt self made projekte petitionen umfragen zeigen dass große teile der gesellschaft bereit sind jetzt wo wir hier reden haben wir vor dem bundestag eine begleitende kundgebung mit über zehn bewegungen und es gibt jetzt gerade redebeiträge und die zeigen ihnen auch wieder dass es nicht nur diese petition ist sondern dass es hunderttausende wenn nicht sogar millionen von menschen sind die tagtäglich immer wieder dafür kämpfen dass hier endlich was passiert im bereich der mobilitäts und verkehrswende zum schluss", 3)

Response status code: 200


('Warum ist das wichtig?', 'Druck für politisches Handeln', 3)

# Generating the qa-pairs

In [14]:
tts = TTS("tts_models/de/thorsten/vits", progress_bar=False)
def to_speech(text, key):
    tts.tts_to_file(text=text, file_path="output.wav")

 > Downloading model to /home/jovyan/.local/share/tts/tts_models--de--thorsten--vits
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [24]:
keys = list(mapping.keys())
     
for i in range(1):
    key = random.choice(keys)
    print(mapping[key][1])
    question, answer, diff = generate_qa(mapping[key][1], 1)
    to_speech(question)
    

dies ist der moment für den so viele frauen jahrzehntelang auf die straße gegangen sind dies ist der moment für den so viele ärztinnen und ärzte gekämpft haben dies ist der moment in dem wir endlich in das parlamentarische verfahren zur streichung von paragraph zweihundertneunzehn a aus dem strafgesetzbuch eintreten dies ist der moment der uns frauen ein stück weit die hoheit über unsere körper zurückgeben wird es ist ein schöner moment
Response status code: 200
 > Text splitted to sentences.
['Welcher Paragraph wird gestrichen?']


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


 > Processing time: 28.678338766098022
 > Real-time factor: 9.684473318311404
